# Computação Gráfica e Realidade Aumentada 
## &copy; 2025
<author>Authors: Guilherme Cabaço e Henrique Gomes</author>
<h1>Assignment 1</h1>

In [ ]:
%%html               
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecshader.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/deecapp.js"></script>
<script src="https://is3l.isr.uc.pt/~pm/CGRA/JS/cgraobject.js"></script>
<script src='https://git.io/glm-js.min.js'></script>

### Vertex e Fragment Shader

In [ ]:
%%html
<script id="my-vertex-shader" type="x-shader/x-vertex">
precision mediump float;

attribute  vec3 in_Position;
attribute  vec3 in_Color;
uniform mat4 MVP;

varying  vec3 ex_Color;

void main(void) {
  
    gl_Position = MVP * vec4(in_Position.x, in_Position.y, in_Position.z, 1.0);

    ex_Color = in_Color;
}
</script>

In [ ]:
%%html
<script id="my-fragment-shader" type="x-shader/x-fragment">
precision mediump float;

varying  vec3 ex_Color;

void main(void) {
  
    gl_FragColor = vec4(ex_Color,1.0);
}
</script>

# A.1
### Cube class

In [ ]:
%%html

<script id="cube">
class cube extends CGRAobject{
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        
        var vertices = [
          -0.5,-0.5,-0.5,  //0
          -0.5,-0.5, 0.5,  //1
          -0.5, 0.5,-0.5,  //2
          -0.5, 0.5, 0.5,  //3
           0.5,-0.5,-0.5,  //4
           0.5,-0.5, 0.5,  //5
           0.5, 0.5,-0.5,  //6
           0.5, 0.5, 0.5   //7
        ];

        var indices = [
          1,5,7, 1,7,3,  // front
          0,2,6, 0,6,4,  // back
          0,1,3, 0,3,2,  // left
          4,6,7, 4,7,5,  // right
          2,3,7, 2,7,6,  // top
          0,4,5, 0,5,1   // bottom
        ];

        var colors = [];
        for (let i = 0; i < vertices.length; i += 3) {//check x of every vertex
            
          //If x==-0.5 assign color1 otherwise its the other side and assign color2  
          const t = (vertices[i] + 0.5);
            
          colors.push(
            color1[0]*(1-t)+color2[0]*t,
            color1[1]*(1-t)+color2[1]*t,
            color1[2]*(1-t)+color2[2]*t
          );
        }
        
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }
    
    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>

### Sphere

In [ ]:
%%html

<script id="sphere">

class sphere extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        this.gl = glcontext;

        var nCircles = 12;  // number of circles
        var nVertices = 12;  // number of vertices per circle
        var radius = 0.5;

        var vertices = [];
        var indices = [];
        var colors = [];

        // Top vertice
        vertices.push(0.0, radius, 0.0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        const topIndex = 0;

        // Generate middle circles
        for (let i = 1; i < nCircles; i++) {
            
            var r = radius * Math.sin(i / nCircles * Math.PI);
            
            var y = radius * Math.cos(i / nCircles * Math.PI);
            
            for (let j = 0; j < nVertices; j++) {
                
                var x = r * Math.cos(j / nVertices * 2 * Math.PI);
                var z = r * Math.sin(j / nVertices * 2 * Math.PI);
                
                vertices.push(x, y, z);
                
                var t = j/nVertices; // based on x coordinate
                colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
                );
            }
        }

        // Bottom vertice
        vertices.push(0.0, -radius, 0.0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        const bottomIndex = vertices.length / 3 - 1;

        // Indices
        // Top 
        for (let j = 1; j < nVertices; j++) {
            const next = (j % nVertices) +1;
            indices.push(topIndex, j, next);
        }

        // Middle stacks
        for (let i = 0; i < nCircles - 2; i++) {
            var start = 1 + i * nVertices;
            var next = start + nVertices;
            
            for (let j = 0; j < nVertices; j++) {
                const k = (j % nVertices) + 1;
            
                indices.push(start + j, next + j, next + k);
                indices.push(start + j, next + k, start + k);
            }
        }

        // Bottom 
        var lastRingStart = 1 + (nCircles - 2) * nVertices;
        for (let j = 0; j < nVertices; j++) {
            const next = (j % nVertices) + 1;
            indices.push(lastRingStart + j, bottomIndex, lastRingStart + next);
        }
        

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}


</script>

### Cylinder

In [ ]:
%%html

<script id="cylinder">
class cylinder extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);

        const n = 12;
        const radius = 0.5;
        const height = 1.0;

        const vertices = [];
        const colors = [];
        const indices = [];

        //Top and base circle vertices
        for (let ring = 0; ring < 2; ring++) {
            var y = ring === 0 ? -height / 2 : height / 2;
            for (let i = 0; i < n; i++) {
                
                var x = radius * Math.cos((i / n) * 2 * Math.PI);
                var z = radius * Math.sin((i / n) * 2 * Math.PI);
                vertices.push(x, y, z);

                var t = i/n; // Gradient along the vertices
                colors.push(
                    color1[0] * (1 - t) + color2[0] * t,
                    color1[1] * (1 - t) + color2[1] * t,
                    color1[2] * (1 - t) + color2[2] * t
                );
            }
        }

        // Top and base center vertices
        var baseIndex = vertices.length / 3;
        vertices.push(0, -height / 2, 0);
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );

        var topIndex = vertices.length / 3;
        vertices.push(0, height / 2, 0);
         colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );


        //Side indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n;
            var bottom1 = i;
            var bottom2 = next;
            var top1 = i + n;
            var top2 = next + n;

            // First triangle
            indices.push(bottom1, top1, bottom2);
            // Second triangle
            indices.push(bottom2, top1, top2);
        }

        // Top indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n; //The result is always the next number after i except in the last where we want 1 again
            indices.push(topIndex, i+n, next+n);
        }
        
        // Base indices
        for (let i = 0; i < n; i++) {
            var next = (i + 1) % n; //The result is always the next number after i except in the last where we want 1 again
            indices.push(baseIndex, next, i);
        }


        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


### Cone

In [ ]:
%%html

<script id="cone">

class cone extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);
        this.gl = glcontext;
        
        var n = 12;
        var radius = 0.5;
        var height = 1.0;
        var vertices = [];
        var indices = [];
        var colors = [];
        
        // top center vertice
        vertices.push(0.0, height/2.0, 0.0);
        
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        
        // base vertices
        for (let i = 0; i < n; i++) {
            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var y = -height/2;
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, y, z);
            
          var t = i/n; //Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
        }
        
        // base center vertice
        vertices.push(0.0, -height/2.0, 0.0);
        
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );
        
        var topIndex = 0;
        var baseIndex = n + 1;
        
        // Indices
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 1 again
            indices.push(topIndex, i, next);
        }
        
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 1 again
            indices.push(baseIndex, next, i);
        }

        
        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }
    
    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }

}

</script>

### Disk

In [ ]:
%%html

<script id="disk">
class disk extends CGRAobject {
    constructor(glcontext, color1=[1,0,0], color2=[0,0,1], n = 12) {
        super(glcontext);

        var radius = 0.5;

        var vertices = [0.0, 0.0, 0.0]; //Center vertex
        
        var colors = [];
        // Center color 
        colors.push(
            (color1[0] + color2[0]) / 2,
            (color1[1] + color2[1]) / 2,
            (color1[2] + color2[2]) / 2
        );

        for (var i = 0; i < n; i++) {
            
            var x = radius * Math.cos((i / n) * 2 * Math.PI);
            var y = 0.0;
            var z = radius * Math.sin((i / n) * 2 * Math.PI);
            vertices.push(x, y, z);

            var t = i/n; // Gradient along the vertices
            colors.push(
                color1[0] * (1 - t) + color2[0] * t,
                color1[1] * (1 - t) + color2[1] * t,
                color1[2] * (1 - t) + color2[2] * t
            );
        }

        var indices = [];
        for (let i = 1; i <= n; i++) {
            var next = (i % n) + 1; //The result is always the next number after i except in the last where we want 1 again
            indices.push(0, i, next);
        }

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
    }

    draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


### Pyramid

In [ ]:
%%html

<script id="pyramid">
class pyramid extends CGRAobject {
      constructor(glcontext, color1=[1,0,0], color2=[0,0,1]) {
        super(glcontext);


        var vertices = [

          -0.5, -0.5, -0.5, // 0
           0.5, -0.5, -0.5, // 1
           0.5, -0.5,  0.5, // 2
          -0.5, -0.5,  0.5, // 3

           0.0,  0.5,  0.0  // 4
        ];

        var indices = [
          // sides
          0, 1, 4, //Left side
          1, 2, 4,
          2, 3, 4,
          3, 0, 4,

          // base
          0, 1, 2,
          0, 2, 3
        ];

        var colors = [];
        for (let i = 0; i < vertices.length; i += 3) {//check x of every vertex

          //If x==-0.5 assign color1 otherwise assign color2  
          const t = (vertices[i] + 0.5);

          colors.push(
            color1[0]*(1-t)+color2[0]*t,
            color1[1]*(1-t)+color2[1]*t,
            color1[2]*(1-t)+color2[2]*t
          );
        }

        this.vertexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.vertexbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(vertices), this.gl.STATIC_DRAW);

        this.colorbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ARRAY_BUFFER, this.colorbuffer);    
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ARRAY_BUFFER, new Float32Array(colors), this.gl.STATIC_DRAW);
        
        this.indexbuffer = this.gl.createBuffer();
        this.gl.bindBuffer(this.gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        // as JS stores everything in 64 bit format and GL expects 32bits...
        this.gl.bufferData(this.gl.ELEMENT_ARRAY_BUFFER, new Uint16Array(indices), this.gl.STATIC_DRAW);

        this.numIndices = indices.length;
  }
    
  draw(viewMat, projMat, shader, parentMat = glm.mat4(1.0)) {
        const gl = this.gl;
        shader.startUsing();

        const mvploc = gl.getUniformLocation(shader.shaderProgram, "MVP");
        const localT = parentMat['*'](this.modelMat);
        const MVP = projMat['*'](viewMat['*'](localT));
        gl.uniformMatrix4fv(mvploc, false, MVP.elements);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.vertexbuffer);
        const posLoc = gl.getAttribLocation(shader.shaderProgram, "in_Position");
        gl.vertexAttribPointer(posLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(posLoc);

        gl.bindBuffer(gl.ARRAY_BUFFER, this.colorbuffer);
        const colLoc = gl.getAttribLocation(shader.shaderProgram, "in_Color");
        gl.vertexAttribPointer(colLoc, 3, gl.FLOAT, false, 0, 0);
        gl.enableVertexAttribArray(colLoc);

        gl.bindBuffer(gl.ELEMENT_ARRAY_BUFFER, this.indexbuffer);
        gl.drawElements(gl.TRIANGLES, this.numIndices, gl.UNSIGNED_SHORT, 0);

        shader.stopUsing();
    }
}
</script>


### Train

In [ ]:
%%html
<script id="locomotiveEngine">

class locomotiveEngine extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1], mode=0){
        super(glcontext);
        
        this.color1 = color1;
        this.color2 = color2;
        
        this.cabine = new cube(this.gl, color1, color2);
        this.lower_body = new cube(this.gl, color1, color2);

        this.engine = new cylinder(this.gl, color1, color2);
        this.chimney = new cylinder(this.gl, color1, color2);
        this.nose = new cylinder(this.gl, color1, color2);
        this.cabine_roof = new cylinder(this.gl, color1, color2);

        this.chimney_top = new cone(this.gl, color1, color2);
        
        this.coolers = [];
        this.swheels = [];
        this.bwheels = [];
        
        this.mode = mode;
        
        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.cabine.setShader(shaderprog);
        this.lower_body.setShader(shaderprog);
        this.engine.setShader(shaderprog);
        this.chimney.setShader(shaderprog);
        this.nose.setShader(shaderprog);
        this.cabine_roof.setShader(shaderprog);
        this.chimney_top.setShader(shaderprog);
        
        for (let c of this.coolers) c.setShader(shaderprog);
        for (let w of this.swheels) w.setShader(shaderprog);
        for (let b of this.bwheels) b.setShader(shaderprog);
    }
    
    create_obj(){
        const c1 = this.color1;
        const c2 = this.color2;
        
        // lower_body
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(6.0, 2.0, 1.0));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        let transform = pos['*'](rot)['*'](scale);
        this.lower_body.setModelTransformation(transform);

        // cabine
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2.0, 1.5, 2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.75, -2.0));
        transform = pos['*'](rot)['*'](scale);
        this.cabine.setModelTransformation(transform);

        // engine
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2.25, 4.0, 2.25));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.5, 1.0));
        transform = pos['*'](rot)['*'](scale);
        this.engine.setModelTransformation(transform);

        // chimney
        rot = glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 1.0, 0.75));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.9, 2.5));
        transform = pos['*'](rot)['*'](scale);
        this.chimney.setModelTransformation(transform);

        // chimney top
        rot = glm.toMat4(glm.angleAxis(glm.radians(180.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.mat4(glm.mat3(0.5));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 3.2, 2.5));
        transform = pos['*'](rot)['*'](scale);
        this.chimney_top.setModelTransformation(transform);

        // small wheels
        const smallWheelPositions = [
            glm.vec3(0.7, -0.25, 2.5),   // right front
            glm.vec3(0.7, -0.25, 1.25),   // right back
            glm.vec3(-0.7, -0.25, 2.5),  // left front
            glm.vec3(-0.7, -0.25, 1.25)   // left back
        ];

        this.swheels = [];

        for (let i = 0; i < smallWheelPositions.length; i++) {
            const isRightSide = smallWheelPositions[i].x > 0;
        
            const color1 = isRightSide ? c1 : c2;
            const color2 = isRightSide ? c2 : c1;
            
            var wheel = new cylinder(this.gl, color1, color2);
            
            if (this.mode==0){
                wheel = new cylinder(this.gl, color1, color2);
            }else if (this.mode==1){
                wheel = new sphere(this.gl, color1, color2);
            }else if (this.mode==2){
                wheel = new cube(this.gl, color1, color2);
            }
            
            const rot = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0, 0.0, 1.0)));
            const scale = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 0.5, 1.0));
            const pos = glm.translate(glm.mat4(1.0), smallWheelPositions[i]);
            const baseTransform = pos['*'](rot)['*'](scale);
            
            wheel.baseTransform = baseTransform;
            
            this.swheels.push(wheel);
        }

        // big wheel        
        const bigWheelPositions = [
            glm.vec3(-0.75, 0.15, -0.5),
            glm.vec3(-0.75, 0.15, -2.25),
            glm.vec3(0.75, 0.15, -0.5),
            glm.vec3(0.75, 0.15, -2.25)
        ];

        this.bwheels = [];

        for (let i = 0; i < bigWheelPositions.length; i++) {
            const isRightSide = bigWheelPositions[i].x > 0;
            
            const color1 = isRightSide ? c1 : c2;
            const color2 = isRightSide ? c2 : c1;
            
            var wheel = new cylinder(this.gl, color1, color2);
            
            if (this.mode==0){
                wheel = new cylinder(this.gl, color1, color2);
            }else if (this.mode==1){
                wheel = new sphere(this.gl, color1, color2);
            }else if (this.mode==2){
                wheel = new cube(this.gl, color1, color2);
            }
            
            rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,0.0,1.0)));
            scale = glm.scale(glm.mat4(1.0), glm.vec3(1.75, 0.5, 1.75));
            const pos = glm.translate(glm.mat4(1.0), bigWheelPositions[i]);
            const baseTransform = pos['*'](rot)['*'](scale);
            
            wheel.baseTransform = baseTransform;
            
            this.bwheels.push(wheel);
        }

        // cabine roof
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.mat4(glm.mat3(2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.3, -2.0));
        transform = pos['*'](rot)['*'](scale);
        this.cabine_roof.setModelTransformation(transform);

        // coolers
        const coolerPositions = [
            glm.vec3(0.6, 0.75, 1.75),
            glm.vec3(-0.6, 0.75, 1.75),
        ];

        this.coolers = [];

        for (let i = 0; i < coolerPositions.length; i++) {
            const cooler = new cylinder(this.gl, c1, c2);
            rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
            scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 2.5, 0.75));
            const pos = glm.translate(glm.mat4(1.0), coolerPositions[i]);
            const transform = pos['*'](rot)['*'](scale);
            cooler.setModelTransformation(transform);
            this.coolers.push(cooler);
        }

        // nose
        rot = glm.toMat4(glm.angleAxis(glm.radians(270.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.75, 0.25, 0.75));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.5, 3.0));
        transform = pos['*'](rot)['*'](scale);
        this.nose.setModelTransformation(transform);
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
        
        this.lower_body.draw(viewM, projectionM, this.shader, parentM);
        this.cabine.draw(viewM, projectionM, this.shader, parentM);
        this.engine.draw(viewM, projectionM, this.shader, parentM);
        this.chimney.draw(viewM, projectionM, this.shader, parentM);
        this.chimney_top.draw(viewM, projectionM, this.shader, parentM);
        this.nose.draw(viewM, projectionM, this.shader, parentM);
        this.cabine_roof.draw(viewM, projectionM, this.shader, parentM);
        
        for (let c of this.coolers) {
            c.draw(viewM, projectionM, this.shader, parentM);
        }
        
        for (let w of this.swheels) {
            const angle = (counter % (2 * Math.PI)) * 2;
            const spin = glm.toMat4(glm.angleAxis(angle, glm.vec3(0.0,1.0,0.0)));
            w.setModelTransformation(w.baseTransform['*'](spin));
            w.draw(viewM, projectionM, this.shader, parentM);
        }
        
        for (let b of this.bwheels) {
            const angle = (counter % (2 * Math.PI)) * 2;
            const spin = glm.toMat4(glm.angleAxis(angle, glm.vec3(0.0,1.0,0.0)));
            b.setModelTransformation(b.baseTransform['*'](spin));
            b.draw(viewM, projectionM, this.shader, parentM);
        }
    }
}


</script>


In [ ]:
%%html
<script id="locomotiveCarriage">

class locomotiveCarriage extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1], carriageSize = 2, mode=0){
        super(glcontext);
        
        this.carriageSize = carriageSize; // 1, 2, 3
        this.color1 = color1;
        this.color2 = color2;
        
        this.body = new cube(this.gl, color1, color2);
        this.lower_body = new cube(this.gl, color1, color2);

        this.roof = new cylinder(this.gl, color1, color2);

        this.swheels = [];
        
        this.mode = mode;
        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.body.setShader(shaderprog);
        this.lower_body.setShader(shaderprog);
        this.roof.setShader(shaderprog);
        
        for (let w of this.swheels) w.setShader(shaderprog);
    }
    
    create_obj(){
        const lengthMap = {1: 3.0, 2: 6.0, 3: 9.0};
        const bodyLength = lengthMap[this.carriageSize] || 6.0;
        
        const c1 = this.color1;
        const c2 = this.color2;
        
        // lower_body
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(bodyLength, 1.0, 1.0));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        let transform = pos['*'](rot)['*'](scale);
        this.lower_body.setModelTransformation(transform);

        // body
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(bodyLength, 2.5, 2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 1.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.body.setModelTransformation(transform);

        
        this.swheels = [];
        let wheelZPositions;
        if(bodyLength === 3.0) wheelZPositions = [1.0, -1.0];
        else if(bodyLength === 6.0) wheelZPositions = [2.5, 1.25, -1.25, -2.5];
        else wheelZPositions = [3.75, 2.5, 1.25, -1.25, -2.5, -3.75];

        for(let z of wheelZPositions){
            for(let side of [-1,1]){ // left/right
                const color1 = side > 0 ? c1 : c2;
                const color2 = side > 0 ? c2 : c1;
                                    
                var wheel = new cylinder(this.gl, color1, color2);
                                    
                if (this.mode==0){
                    wheel = new cylinder(this.gl, color1, color2);
                }else if (this.mode==1){
                    wheel = new sphere(this.gl, color1, color2);
                }else if (this.mode==2){
                    wheel = new cube(this.gl, color1, color2);
                }

                const rotW = glm.toMat4(glm.angleAxis(glm.radians(90.0), glm.vec3(0.0, 0.0, 1.0)));
                const scaleW = glm.scale(glm.mat4(1.0), glm.vec3(1.0, 0.5, 1.0));
                const posW = glm.translate(glm.mat4(1.0), glm.vec3(0.7*side, -0.25, z));
                wheel.baseTransform = posW['*'](rotW)['*'](scaleW);

                this.swheels.push(wheel);
            }
        }

        // roof
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(1.0,0.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(2.0, bodyLength, 2.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 2.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.roof.setModelTransformation(transform);

    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
        
        this.lower_body.draw(viewM, projectionM, this.shader, parentM);
        this.body.draw(viewM, projectionM, this.shader, parentM);
        this.roof.draw(viewM, projectionM, this.shader, parentM);
        
        for (let w of this.swheels) {
            const angle = (counter % (2 * Math.PI)) * 2;
            const spin = glm.toMat4(glm.angleAxis(angle, glm.vec3(0.0,1.0,0.0)));
            w.setModelTransformation(w.baseTransform['*'](spin));
            w.draw(viewM, projectionM, this.shader, parentM);
        }
        
    }
}


</script>

In [ ]:
%%html
<script id="train">

class train extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.head = new locomotiveEngine(this.gl, color1, color2);
        this.c1 = new locomotiveCarriage(this.gl, color1, color2, 3);
        this.c2 = new locomotiveCarriage(this.gl, color1, color2, 1);
        this.c3 = new locomotiveCarriage(this.gl, color1, color2, 2);

        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.head.setShader(shaderprog);
        this.c1.setShader(shaderprog);
        this.c2.setShader(shaderprog);
        this.c3.setShader(shaderprog);
    }
    
    create_obj(){
        this.headOffset = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, 0.0));
        this.c1Offset = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, -8.5));
        this.c2Offset = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, -15.5));
        this.c3Offset = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.0, -21.0));
    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0), counter = 0){
        
        this.head.draw(viewM, projectionM, parentM['*'](this.headOffset), counter);

        this.c1.draw(viewM, projectionM, parentM['*'](this.c1Offset), counter);
        this.c2.draw(viewM, projectionM, parentM['*'](this.c2Offset), counter);
        this.c3.draw(viewM, projectionM, parentM['*'](this.c3Offset), counter);
    }
}


</script>

### Sphynx

In [ ]:
%%html
<script id="sphynx">

class sphynx extends CGRAobject {
    constructor(glcontext, color1 = [1,0,0], color2 = [0, 0, 1]){
        super(glcontext);
        
        this.color1 = color1;
        this.color2 = color2;
        
        
        this.floor = new cube(this.gl, color1, color2);
        this.ceiling = new cube(this.gl, color1, color2);
        
        this.backwall = new cube(this.gl, color1, color2);
        
        this.leftwall = new cube(this.gl, color1, color2);
        this.leftwall1 = new cube(this.gl, color1, color2);
        this.leftwall2 = new cube(this.gl, color1, color2);
        this.leftwall3 = new cube(this.gl, color1, color2);
        this.rightwall = new cube(this.gl, color1, color2);
        this.rightwall1 = new cube(this.gl, color1, color2);
        this.rightwall2 = new cube(this.gl, color1, color2);
        this.rightwall3 = new cube(this.gl, color1, color2);
        
        
        this.leftpaw_back = new cylinder(this.gl, color1, color2);
        this.rightpaw_back = new cylinder(this.gl, color1, color2);
        
        
        this.leftpaw_front1 = new cylinder(this.gl, color1, color2);
        this.leftpaw_front2 = new cylinder(this.gl, color1, color2);
        this.leftpaw_front3 = new cylinder(this.gl, color1, color2);
        this.leftpaw_front4 = new cylinder(this.gl, color1, color2);
        
        this.rightpaw_front1 = new cylinder(this.gl, color1, color2);
        this.rightpaw_front2 = new cylinder(this.gl, color1, color2);
        this.rightpaw_front3 = new cylinder(this.gl, color1, color2);
        this.rightpaw_front4 = new cylinder(this.gl, color1, color2);
        
        this.frontwall_top = new cube(this.gl, color1, color2);
        this.frontwall_left = new cube(this.gl, color1, color2);
        this.frontwall_right = new cube(this.gl, color1, color2);
        

        this.create_obj();
    }
    
    setShader(shaderprog){
        this.shader = shaderprog;
        
        this.floor.setShader(shaderprog);
        this.ceiling.setShader(shaderprog);
        this.backwall.setShader(shaderprog);

        this.leftwall.setShader(shaderprog);
        this.leftwall1.setShader(shaderprog);
        this.leftwall2.setShader(shaderprog);
        this.leftwall3.setShader(shaderprog);
        this.rightwall.setShader(shaderprog);
        this.rightwall1.setShader(shaderprog);
        this.rightwall2.setShader(shaderprog);
        this.rightwall3.setShader(shaderprog);

        this.frontwall_top.setShader(shaderprog);
        this.frontwall_left.setShader(shaderprog);
        this.frontwall_right.setShader(shaderprog);

        this.leftpaw_back.setShader(shaderprog);
        this.rightpaw_back.setShader(shaderprog);

        this.leftpaw_front1.setShader(shaderprog);
        this.leftpaw_front2.setShader(shaderprog);
        this.leftpaw_front3.setShader(shaderprog);
        this.leftpaw_front4.setShader(shaderprog);

        this.rightpaw_front1.setShader(shaderprog);
        this.rightpaw_front2.setShader(shaderprog);
        this.rightpaw_front3.setShader(shaderprog);
        this.rightpaw_front4.setShader(shaderprog);
        
    }
    
    create_obj(){
        
        const c1 = this.color1;
        const c2 = this.color2;
        
        //Floor
        let scale = glm.scale(glm.mat4(1.0), glm.vec3(21, 0.1, 43.9));
        let pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.05, -8.0));
        let transform = pos['*'](scale);
        this.floor.setModelTransformation(transform);
        
        //Ceiling
        scale = glm.scale(glm.mat4(1.0), glm.vec3(21, 0.2, 60.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 6.5 , 0.0));
        transform = pos['*'](scale);
        this.ceiling.setModelTransformation(transform);
        
        //Backwall
        scale = glm.scale(glm.mat4(1.0), glm.vec3(21, 6.4, 8.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 3.2, 26.0));
        transform = pos['*'](scale);
        this.backwall.setModelTransformation(transform);

        //Leftwall 
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.2, 2, 43.9));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(10.4, 1, -8.0));
        transform = pos['*'](scale);
        this.leftwall.setModelTransformation(transform);
        
        //Leftwall 1
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.2, 2.4, 43.9));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(10.4, 5.2, -8.0));
        transform = pos['*'](scale);
        this.leftwall1.setModelTransformation(transform);
        
        //Leftwall 2
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.2, 6.4, 6));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(10.4, 3.2, 11.0));
        transform = pos['*'](scale);
        this.leftwall2.setModelTransformation(transform);
        
        //Leftwall 3
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.2, 6.4, 6));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(10.4, 3.2, -11.0));
        transform = pos['*'](scale);
        this.leftwall3.setModelTransformation(transform);
        
        //Rightwall 
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.2, 2, 43.9));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-10.4, 1, -8.0));
        transform = pos['*'](scale);
        this.rightwall.setModelTransformation(transform);
        
        //Rightwall 1
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.2, 2.4, 43.9));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-10.4, 5.2, -8.0));
        transform = pos['*'](scale);
        this.rightwall1.setModelTransformation(transform);
        
        //Rightwall 2
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.2, 6.4, 6));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-10.4, 3.2, 11.0));
        transform = pos['*'](scale);
        this.rightwall2.setModelTransformation(transform);
        
        //Rightwall 3
        scale = glm.scale(glm.mat4(1.0), glm.vec3(0.2, 6.4, 6));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-10.4, 3.2, -11.0));
        transform = pos['*'](scale);
        this.rightwall3.setModelTransformation(transform);

        //Frontwall top
        scale = glm.scale(glm.mat4(1.0), glm.vec3(21, 4.4, 0.2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 4.2, -29.9));
        transform = pos['*'](scale);
        this.frontwall_top.setModelTransformation(transform);
        
        //Frontwall left
        scale = glm.scale(glm.mat4(1.0), glm.vec3(8.5, 2.0, 0.2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(-6.250, 1, -29.9));
        transform = pos['*'](scale);
        this.frontwall_left.setModelTransformation(transform);
        
        //frontwall right
        scale = glm.scale(glm.mat4(1.0), glm.vec3(8.5, 2.0, 0.2));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(6.250, 1, -29.9));
        transform = pos['*'](scale);
        this.frontwall_right.setModelTransformation(transform);
        
        
        
        

        //leftpaw back
        let rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_back.setModelTransformation(transform);
        
        //rightpaw right
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_back.setModelTransformation(transform);

        //Leftpaw front 1
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front1.setModelTransformation(transform);
        
        //Leftpaw front 2
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front2.setModelTransformation(transform);
        
        //Leftpaw front 3
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front3.setModelTransformation(transform);
        
        //Leftpaw front 4
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.leftpaw_front4.setModelTransformation(transform);

        //Rightpaw front 1
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front1.setModelTransformation(transform);
        
        //Rightpaw front 2
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front2.setModelTransformation(transform);
        
        //Rightpaw front 3
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front3.setModelTransformation(transform);
        
        //Rightpaw front 4
        rot = glm.toMat4(glm.angleAxis(glm.radians(90.0),glm.vec3(0.0,1.0,0.0)));
        scale = glm.scale(glm.mat4(1.0), glm.vec3(1, 1.0, 1.0));
        pos = glm.translate(glm.mat4(1.0), glm.vec3(0.0, 0.5, 0.0));
        transform = pos['*'](rot)['*'](scale);
        this.rightpaw_front4.setModelTransformation(transform);

    }
    
    draw(viewM = glm.mat4(1.0), projectionM = glm.mat4(1.0), parentM = glm.mat4(1.0)){
        
        this.floor.draw(viewM, projectionM, this.shader, parentM);
        this.ceiling.draw(viewM, projectionM, this.shader, parentM);
        this.backwall.draw(viewM, projectionM, this.shader, parentM);

        this.leftwall.draw(viewM, projectionM, this.shader, parentM);
        this.leftwall1.draw(viewM, projectionM, this.shader, parentM);
        this.leftwall2.draw(viewM, projectionM, this.shader, parentM);
        this.leftwall3.draw(viewM, projectionM, this.shader, parentM);
        this.rightwall.draw(viewM, projectionM, this.shader, parentM);
        this.rightwall1.draw(viewM, projectionM, this.shader, parentM);
        this.rightwall2.draw(viewM, projectionM, this.shader, parentM);
        this.rightwall3.draw(viewM, projectionM, this.shader, parentM);

        this.frontwall_top.draw(viewM, projectionM, this.shader, parentM);
        this.frontwall_left.draw(viewM, projectionM, this.shader, parentM);
        this.frontwall_right.draw(viewM, projectionM, this.shader, parentM);

        this.leftpaw_back.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_back.draw(viewM, projectionM, this.shader, parentM);

        this.leftpaw_front1.draw(viewM, projectionM, this.shader, parentM);
        this.leftpaw_front2.draw(viewM, projectionM, this.shader, parentM);
        this.leftpaw_front3.draw(viewM, projectionM, this.shader, parentM);
        this.leftpaw_front4.draw(viewM, projectionM, this.shader, parentM);

        this.rightpaw_front1.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_front2.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_front3.draw(viewM, projectionM, this.shader, parentM);
        this.rightpaw_front4.draw(viewM, projectionM, this.shader, parentM);
    }
}


</script>

# Main

In [ ]:
%%html
<canvas id="myCanvas2" width="800" height="800" style="border:2px solid #000000;">
      Error: Your browser does not support the HTML canvas tag.
</canvas>
    
<script id="myapp">

class myapp2 extends DEECapp{
    
    orientation = 0; // orientation of the camera
    mode = 0; // Visitor == 0 or Train == 1
    shape = 0; //Shape for the train wheels
    
    track_length = 70*2;
    track_width = 35*2;
    
    loopSpeed = 5.0;
    loopDuration = this.loopSpeed*2*Math.PI;  
    pauseDuration = 3.0;
    
    cameraPos = glm.vec3(0, 1, -150); //Initial position of the camera
    
    initialize(){
        // no call to super.initialize() this time because
        // in this case we don't want to use the default shader.
        // So we need to explicitly prepare it... and we may indeed prepare 
        // as many shaderprograms as we want.
        
        var fragsrc = document.getElementById("my-fragment-shader").text;
        var vertsrc = document.getElementById("my-vertex-shader").text;
        
        this.shaderprog = new DEECshader(this.gl);
        this.shaderprog.srcShaders(vertsrc,fragsrc);
        
        // perform other initializations
        this.gl.enable(this.gl.DEPTH_TEST);
        this.gl.clearColor(1.0,1.0,1.0,1.0);
        
        
        //===========================================================
        // Initialize objects
        
        //Sphynx train station
        this.sphynx = new sphynx(this.gl, [0.800,0.570,0.0320], [1,1,1]);
        this.sphynx.setShader(this.shaderprog);
        
        //Pyramids
        this.bigpyramid1 = new pyramid(this.gl, [0.800,0.570,0.0320], [0.800,0.570,0.0320]);
        this.bigpyramid1.setShader(this.shaderprog);
        
        this.bigpyramid2 = new pyramid(this.gl, [0.800,0.570,0.0320], [0.800,0.570,0.0320]);
        this.bigpyramid2.setShader(this.shaderprog);
        
        this.bigpyramid3 = new pyramid(this.gl, [0.800,0.570,0.0320], [0.800,0.570,0.0320]);
        this.bigpyramid3.setShader(this.shaderprog);
        
        
        //Sun
        this.sun = new sphere(this.gl, [0.970,0.558,0.0194], [0.970,0.558,0.0194]);
        this.sun.setShader(this.shaderprog);
        
        
        //Random Objects
        this.cone = new cone(this.gl);
        this.cone.setShader(this.shaderprog);
        
        this.disk = new disk(this.gl);
        this.disk.setShader(this.shaderprog);
        
        this.cube = new cube(this.gl);
        this.cube.setShader(this.shaderprog);
        
        this.cylinder = new cylinder(this.gl);
        this.cylinder.setShader(this.shaderprog);
        
        this.sphere = new sphere(this.gl);
        this.sphere.setShader(this.shaderprog);
        
        //Floor and track
        this.floor = new disk(this.gl, [0.950,0.816,0.333], [0.950,0.816,0.333]);
        this.floor.setShader(this.shaderprog);
        
        this.inner_floor = new disk(this.gl, [0.950,0.816,0.333], [0.950,0.816,0.333], 60);
        this.inner_floor.setShader(this.shaderprog);
        
        this.track = new disk(this.gl, [0.6,0.6,0.6], [0.6,0.6,0.6], 60);
        this.track.setShader(this.shaderprog);
        
        //Train
        this.train = new locomotiveEngine(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1]);
        this.train.setShader(this.shaderprog);
        
        this.train_carriage_1 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],2);
        this.train_carriage_1.setShader(this.shaderprog);
        
        this.train_carriage_2 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],1);
        this.train_carriage_2.setShader(this.shaderprog);
        
        this.train_carriage_3 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],3);
        this.train_carriage_3.setShader(this.shaderprog);
        //===========================================================
        
        //===========================================================
        //Translations
        this.trans_bigpyramid1 = glm.translate(glm.mat4(1.0),glm.vec3(700,27,650));
        this.trans_bigpyramid2 = glm.translate(glm.mat4(1.0),glm.vec3(500,37,550));
        this.trans_bigpyramid3 = glm.translate(glm.mat4(1.0),glm.vec3(300,17,450));
        
        this.trans_sun = glm.translate(glm.mat4(1.0),glm.vec3(400,200,700));
        
        //Random Objects
        this.trans_cone = glm.translate(glm.mat4(1.0),glm.vec3(0,1.5,19*2));
        this.trans_disk = glm.translate(glm.mat4(1.0),glm.vec3(36*2,1.2,0));
        this.trans_cube = glm.translate(glm.mat4(1.0),glm.vec3(0,1.5,-19*2));
        this.trans_cylinder = glm.translate(glm.mat4(1.0),glm.vec3(-36*2,1.5,0));
        this.trans_sphere = glm.translate(glm.mat4(1.0),glm.vec3(0,0.5,0));
        
        //Floor and track
        this.trans_track = glm.translate(glm.mat4(1.0),glm.vec3(0,0.01,0));
        this.trans_inner_floor = glm.translate(glm.mat4(1.0),glm.vec3(0,0.02,0));
        
        //Sphynx
        this.trans_sphynx = glm.translate(glm.mat4(1.0),glm.vec3(0,0,-50));
        //===========================================================
        
        //===========================================================
        //Scales
        this.floor_scale = glm.scale(glm.mat4(1.0),glm.vec3(1000,0,1000));
        this.inner_floor_scale = glm.scale(glm.mat4(1.0),glm.vec3(this.track_length-8,0,this.track_width-8));
        
        this.track_scale = glm.scale(glm.mat4(1.0),glm.vec3(this.track_length,0,this.track_width));
        
        this.sun_scale = glm.scale(glm.mat4(1.0),glm.vec3(50,50,50));
        
        this.bigpyramid1_scale = glm.scale(glm.mat4(1.0),glm.vec3(100,75,100));
        this.bigpyramid2_scale = glm.scale(glm.mat4(1.0),glm.vec3(150,115,150));
        this.bigpyramid3_scale = glm.scale(glm.mat4(1.0),glm.vec3(100,75,100));
        //===========================================================
        
        this.projectionM = glm.perspective(glm.radians(45),1,0.1,1000);
        this.viewM = glm.lookAt(this.cameraPos,glm.vec3(this.cameraPos.x+Math.sin(glm.radians(this.orientation)),1,this.cameraPos.z+Math.cos(glm.radians(this.orientation))),glm.vec3(0,1,0));
        
        this.startTime = Date.now(); 
  
        window.addEventListener("keypress",(evt)=>this.keyprocess(evt),false);
    }
    
    keyprocess(evt){
        var step=1;
        console.log('Key pressed. Keycode:'+evt.keyCode + ' Char: \'' + String.fromCharCode(evt.charCode)+'\'');
         switch (evt.keyCode) {
             case 27:
                alert("You pressed the \"Escape\" key."); 
                break;
             case 118: // v
                 if(this.mode == 0){
                     this.mode = 1; //Change to Passenger
                 }
                 else if(this.mode == 1){
                     this.mode = 0; //Change to Visitor
                 }
                 break;
             
             case 119: // w
                 if (this.mode == 0){
                     this.cameraPos.x += step*Math.sin(glm.radians(this.orientation));
                     this.cameraPos.z += step*Math.cos(glm.radians(this.orientation));
                 }
                 break;
             
             case 115: //s
                 if (this.mode == 0){
                     this.cameraPos.x -= step*Math.sin(glm.radians(this.orientation));
                     this.cameraPos.z -= step*Math.cos(glm.radians(this.orientation));
                }
                 break;
             
             case 97: //
                 if (this.mode == 0){
                     this.orientation+=2;
                 }
                 break;
             
             case 100: //d
                 if (this.mode == 0){
                     this.orientation-=2;
                 }
                 break;
             
             case 32: //switch shape
                 if (this.shape == 0){
                    this.shape = 1;
                    this.train = new locomotiveEngine(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],this.shape);
                    this.train.setShader(this.shaderprog);
                     
                    this.train_carriage_1 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],2,this.shape);
                    this.train_carriage_1.setShader(this.shaderprog);

                    this.train_carriage_2 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],1,this.shape);
                    this.train_carriage_2.setShader(this.shaderprog);

                    this.train_carriage_3 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],3,this.shape);
                    this.train_carriage_3.setShader(this.shaderprog);
                 }
                 else if (this.shape == 1){
                     this.shape = 2;
                     this.train = new locomotiveEngine(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],this.shape);
                     this.train.setShader(this.shaderprog);
                     
                     this.train_carriage_1 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],2,this.shape);
                     this.train_carriage_1.setShader(this.shaderprog);

                     this.train_carriage_2 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],1,this.shape);
                     this.train_carriage_2.setShader(this.shaderprog);

                     this.train_carriage_3 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],3,this.shape);
                     this.train_carriage_3.setShader(this.shaderprog);
                     
                 }
                 else if (this.shape == 2){
                     this.shape = 0;
                     this.train = new locomotiveEngine(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],this.shape);
                     this.train.setShader(this.shaderprog);
                     
                     this.train_carriage_1 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],2,this.shape);
                     this.train_carriage_1.setShader(this.shaderprog);

                     this.train_carriage_2 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],1,this.shape);
                     this.train_carriage_2.setShader(this.shaderprog);

                     this.train_carriage_3 = new locomotiveCarriage(this.gl, [0.0, 0.0, 0.0], [0, 0.0, 1],3,this.shape);
                     this.train_carriage_3.setShader(this.shaderprog);
                 }
             
                 theta = -2.13*Math.PI/4;
        
        var dx = -1/this.loopSpeed*(this.track_length/2-2) * Math.sin(1/this.loopSpeed*t+theta);
        var dz = 1/this.loopSpeed*(this.track_width/2-2) * Math.cos(1/this.loopSpeed*t+theta);
        
        trainRot = glm.toMat4(glm.angleAxis(Math.atan2(dx,dz), glm.vec3(0,1,0))); 
        trainPos = glm.translate(glm.mat4(1.0), glm.vec3((this.track_length/2-2) * Math.cos(1/this.loopSpeed*t+theta), 0.5, (this.track_width/2-2) * Math.sin(1/this.loopSpeed*t+theta)));
        modelM = trainPos['*'](trainRot);
        this.train_carriage_1.setModelTransformation(modelM);
                                                      
        this.train_carriage_1.draw(this.viewM, this.projectionM, modelM, -t);
        
        
        
        theta = -2.24*Math.PI/4;
        
        var dx = -1/this.loopSpeed*(this.track_length/2-2) * Math.sin(1/this.loopSpeed*t+theta);
        var dz = 1/this.loopSpeed*(this.track_width/2-2) * Math.cos(1/this.loopSpeed*t+theta);
        
        trainRot = glm.toMat4(glm.angleAxis(Math.atan2(dx,dz), glm.vec3(0,1,0))); 
        trainPos = glm.translate(glm.mat4(1.0), glm.vec3((this.track_length/2-2) * Math.cos(1/this.loopSpeed*t+theta), 0.5, (this.track_width/2-2) * Math.sin(1/this.loopSpeed*t+theta)));
        modelM = trainPos['*'](trainRot);
        this.train_carriage_2.setModelTransformation(modelM);
                                                      
        this.train_carriage_2.draw(this.viewM, this.projectionM, modelM, -t);
        
        
        
        theta = -2.38*Math.PI/4;
        
        var dx = -1/this.loopSpeed*(this.track_length/2-2) * Math.sin(1/this.loopSpeed*t+theta);
        var dz = 1/this.loopSpeed*(this.track_width/2-2) * Math.cos(1/this.loopSpeed*t+theta);
        
        trainRot = glm.toMat4(glm.angleAxis(Math.atan2(dx,dz), glm.vec3(0,1,0))); 
        trainPos = glm.translate(glm.mat4(1.0), glm.vec3((this.track_length/2-2) * Math.cos(1/this.loopSpeed*t+theta), 0.5, (this.track_width/2-2) * Math.sin(1/this.loopSpeed*t+theta)));
        modelM = trainPos['*'](trainRot);
        this.train_carriage_3.setModelTransformation(modelM);
                                                      
        this.train_carriage_3.draw(this.viewM, this.projectionM, modelM, -t);
                 break;
         }
    }
    
    render(){
       
        this.gl.clear(this.gl.COLOR_BUFFER_BIT);
        
        //Background color
        this.gl.clearColor(0, 0, 1, 0.2);
        
        
        //Calculate time cycle and view matrix based on mode 
        var t = (Date.now()-this.startTime)/1000.0;  
        var cycleTime = (t % (this.loopDuration + this.pauseDuration));

        if (cycleTime < this.pauseDuration) {
           var t = 0;
        } else {
           t = cycleTime - this.pauseDuration;
        }

        var x = (this.track_length/2) * Math.cos(1/this.loopSpeed*t-Math.PI/2);
        var z = (this.track_width/2) * Math.sin(1/this.loopSpeed*t-Math.PI/2);
        
        if(this.mode == 1){   
            this.viewM = glm.lookAt(glm.vec3(x,1,z),glm.vec3(x+Math.cos(1/this.loopSpeed*t-Math.PI/2),1,z+Math.sin(1/this.loopSpeed*t-Math.PI/2)),glm.vec3(0,1,0));
        }
        else if(this.mode == 0){
            this.viewM = glm.lookAt(this.cameraPos,glm.vec3(this.cameraPos.x+Math.sin(glm.radians(this.orientation)),1,this.cameraPos.z+Math.cos(glm.radians(this.orientation))),glm.vec3(0,1,0));
        }
        
        //===========================================================
        //Track and floor
        this.sphynx.setModelTransformation(this.trans_sphynx);
        this.sphynx.draw(this.viewM, this.projectionM, this.trans_sphynx);
        
        this.floor.setModelTransformation(this.floor_scale);
        this.floor.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.inner_floor.setModelTransformation(this.trans_inner_floor['*'](this.inner_floor_scale));
        this.inner_floor.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.track.setModelTransformation(this.trans_track['*'](this.track_scale));
        this.track.draw(this.viewM, this.projectionM, this.shaderprog);
        
        
        //Pyramids
        this.bigpyramid1.setModelTransformation(this.trans_bigpyramid1['*'](this.bigpyramid1_scale));
        this.bigpyramid1.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.bigpyramid2.setModelTransformation(this.trans_bigpyramid2['*'](this.bigpyramid2_scale));
        this.bigpyramid2.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.bigpyramid3.setModelTransformation(this.trans_bigpyramid3['*'](this.bigpyramid3_scale));
        this.bigpyramid3.draw(this.viewM, this.projectionM, this.shaderprog);
        
        
        //Sun 
        this.sun.setModelTransformation(this.trans_sun['*'](this.sun_scale));
        this.sun.draw(this.viewM, this.projectionM, this.shaderprog);
        
        
        //Random Objects
        this.cone.setModelTransformation(this.trans_cone);
        this.cone.draw(this.viewM, this.projectionM, this.shaderprog);
           
        this.disk.setModelTransformation(this.trans_disk);
        this.disk.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.cube.setModelTransformation(this.trans_cube);
        this.cube.draw(this.viewM, this.projectionM, this.shaderprog);

        this.cylinder.setModelTransformation(this.trans_cylinder);
        this.cylinder.draw(this.viewM, this.projectionM, this.shaderprog);
        
        this.sphere.setModelTransformation(this.trans_sphere);
        this.sphere.draw(this.viewM, this.projectionM, this.shaderprog);
        
        
        //Train Engine    
        var theta = -Math.PI/2;
        
        var dx = -1/this.loopSpeed*(this.track_length/2-2) * Math.sin(1/this.loopSpeed*t+theta);
        var dz = 1/this.loopSpeed*(this.track_width/2-2) * Math.cos(1/this.loopSpeed*t+theta);
        
        var trainRot = glm.toMat4(glm.angleAxis(Math.atan2(dx,dz), glm.vec3(0,1,0))); 
        var trainPos = glm.translate(glm.mat4(1.0), glm.vec3((this.track_length/2-2) * Math.cos(1/this.loopSpeed*t+theta), 0.5, (this.track_width/2-2) * Math.sin(1/this.loopSpeed*t+theta)));
        var modelM = trainPos['*'](trainRot);
        this.train.setModelTransformation(modelM);
                                                      
        this.train.draw(this.viewM, this.projectionM, modelM, -t);
        
        
        //Train Carriage 1
        theta = -2.13*Math.PI/4;
        
        var dx = -1/this.loopSpeed*(this.track_length/2-2) * Math.sin(1/this.loopSpeed*t+theta);
        var dz = 1/this.loopSpeed*(this.track_width/2-2) * Math.cos(1/this.loopSpeed*t+theta);
        
        trainRot = glm.toMat4(glm.angleAxis(Math.atan2(dx,dz), glm.vec3(0,1,0))); 
        trainPos = glm.translate(glm.mat4(1.0), glm.vec3((this.track_length/2-2) * Math.cos(1/this.loopSpeed*t+theta), 0.5, (this.track_width/2-2) * Math.sin(1/this.loopSpeed*t+theta)));
        modelM = trainPos['*'](trainRot);
        this.train_carriage_1.setModelTransformation(modelM);
                                                      
        this.train_carriage_1.draw(this.viewM, this.projectionM, modelM, -t);
        
        
        //Train Carriage 2
        theta = -2.24*Math.PI/4;
        
        var dx = -1/this.loopSpeed*(this.track_length/2-2) * Math.sin(1/this.loopSpeed*t+theta);
        var dz = 1/this.loopSpeed*(this.track_width/2-2) * Math.cos(1/this.loopSpeed*t+theta);
        
        trainRot = glm.toMat4(glm.angleAxis(Math.atan2(dx,dz), glm.vec3(0,1,0))); 
        trainPos = glm.translate(glm.mat4(1.0), glm.vec3((this.track_length/2-2) * Math.cos(1/this.loopSpeed*t+theta), 0.5, (this.track_width/2-2) * Math.sin(1/this.loopSpeed*t+theta)));
        modelM = trainPos['*'](trainRot);
        this.train_carriage_2.setModelTransformation(modelM);
                                                      
        this.train_carriage_2.draw(this.viewM, this.projectionM, modelM, -t);
        
        
        //Train Carriage 3
        theta = -2.38*Math.PI/4;
        
        var dx = -1/this.loopSpeed*(this.track_length/2-2) * Math.sin(1/this.loopSpeed*t+theta);
        var dz = 1/this.loopSpeed*(this.track_width/2-2) * Math.cos(1/this.loopSpeed*t+theta);
        
        trainRot = glm.toMat4(glm.angleAxis(Math.atan2(dx,dz), glm.vec3(0,1,0))); 
        trainPos = glm.translate(glm.mat4(1.0), glm.vec3((this.track_length/2-2) * Math.cos(1/this.loopSpeed*t+theta), 0.5, (this.track_width/2-2) * Math.sin(1/this.loopSpeed*t+theta)));
        modelM = trainPos['*'](trainRot);
        this.train_carriage_3.setModelTransformation(modelM);
                                                      
        this.train_carriage_3.draw(this.viewM, this.projectionM, modelM, -t);
        //===========================================================

    }
    
}


var app2 = new myapp2('myCanvas2');

app2.run();

</script>